In [ ]:
import os
import json

from IPython.display import display, Markdown

import numpy as np
import matplotlib.pyplot as plt

from pymedphys.level1.msqconnect import mosaiq_connect
from pymedphys.level1.configutilities import (
    get_cache_filepaths, get_mu_density_parameters,
    get_index, get_centre, get_sql_servers, get_sql_servers_list,
    get_filepath
)
from pymedphys.level1.mudensity import find_relevant_control_points, calc_mu_density
from pymedphys.level2.msqdelivery import multi_fetch_and_verify_mosaiq
from pymedphys.level3.analyselogfiles import (
    analyse_single_hash, find_consecutive_logfiles, get_field_id_key_map, plot_results,
    calc_comparison
)
from pymedphys.level4.comparebygantry import (
    group_consecutive_logfiles, assert_array_agreement, get_gantry_tolerance,
    get_logfile_delivery_data_bygantry, get_logfile_mu_density_bygantry,
    get_mosaiq_delivery_data_bygantry, get_mosaiq_mu_density_bygantry,
    comparison_results
)

from decode_trf.decode_trf import delivery_data_from_logfile

In [ ]:
with open('../config.json') as config_file:
    config = json.load(config_file)

In [ ]:
index = get_index(config)
file_hashes = np.array(list(index.keys()))

field_types = np.array([
    index[file_hash]['delivery_details']['field_type']
    for file_hash in file_hashes
])

file_hashes = file_hashes[field_types == 'DMLC']

is_qa = np.array([
    index[file_hash]['delivery_details']['qa_mode']
    for file_hash in file_hashes
])

file_hashes = file_hashes[np.invert(is_qa)]

machine = np.array([
    index[file_hash]['logfile_header']['machine']
    for file_hash in file_hashes
])

# limit to RCCC for now
file_hashes = file_hashes[(machine == '2619') | (machine == '2694')]
np.random.shuffle(file_hashes)

# might need course information
index[file_hashes[0]]

In [ ]:
patient_grouped_fields = dict()
field_id_grouped_hashes = dict()

for file_hash in file_hashes:
    delivery_details = index[file_hash]['delivery_details']
    patient_id = delivery_details['patient_id']
    field_id = delivery_details['field_id']
    
    if patient_id not in patient_grouped_fields:
        patient_grouped_fields[patient_id] = set()
        
    patient_grouped_fields[patient_id].add(field_id)
    
    if field_id not in field_id_grouped_hashes:
        field_id_grouped_hashes[field_id] = []
        
    field_id_grouped_hashes[field_id].append(file_hash)

In [ ]:
patient_ids = list(patient_grouped_fields.keys())

In [ ]:
patient_id = '013340'
fields = patient_grouped_fields[patient_id]

In [ ]:
patient_id

In [ ]:
for field in fields:
    print(field_id_grouped_hashes[field])

In [ ]:
with mosaiq_connect('msqsql') as cursor:
    for field in fields:
        mosaiq_delivery_data = multi_fetch_and_verify_mosaiq(
            cursor, field)
        print(set(mosaiq_delivery_data.gantry))

In [ ]:
######
# TODO Need to expand this out to run over all fields
######
field_id = list(fields)[0]
keys = np.array(field_id_grouped_hashes[field_id])

logfile_groups = group_consecutive_logfiles(keys, index)
logfile_groups = [
    tuple(group)
    for group in logfile_groups
]
logfile_groups = [logfile_groups[1]]
logfile_groups

In [ ]:
with mosaiq_connect('msqsql') as cursor:
    mosaiq_delivery_data = multi_fetch_and_verify_mosaiq(
        cursor, field_id)

In [ ]:
mosaiq_gantry_angles = np.unique(mosaiq_delivery_data.gantry)
mosaiq_gantry_angles

In [ ]:
logfile_delivery_data_bygantry = get_logfile_delivery_data_bygantry(index, config, logfile_groups, mosaiq_gantry_angles)

In [ ]:
logfile_mu_density_bygantry = get_logfile_mu_density_bygantry(logfile_groups, mosaiq_gantry_angles, logfile_delivery_data_bygantry)

In [ ]:
mosaiq_delivery_data_bygantry = get_mosaiq_delivery_data_bygantry(mosaiq_delivery_data)
# mosaiq_delivery_data_bygantry

In [ ]:
mosaiq_mu_density_bygantry = get_mosaiq_mu_density_bygantry(mosaiq_delivery_data_bygantry)

In [ ]:
comparison_results = comparison_results(mosaiq_mu_density_bygantry, logfile_mu_density_bygantry)
comparison_results